In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import datetime as dt
import random

In [19]:
#환경부 데이터 모으기

#드라이버 연결, url 설정
driver=webdriver.Edge("c:/msedgedriver") 
url="https://www.ev.or.kr/evmonitor#"

#웹사이트 이동
driver.get(url)
time.sleep(3)

seoul_list=['강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구',\
           '마포구','서대문구','서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구']
type_list=['공공시설','주차시설','휴게시설','관광시설','상업시설','차량정비시설','공동주택시설','근린생활시설','교육문화시설','기타시설']
title=[]
state=[]
kind=[]
date=[]
gu=[]
facility=[]

driver.find_element_by_id("F_SI_DO1").send_keys('서울특별시') #서울특별시 입력
time.sleep(1)

for i in seoul_list:
    driver.find_element_by_id("F_GU_GUN1").send_keys(i) #구입력
    time.sleep(1)
    for j in type_list:
        driver.find_element_by_id('F_STAT_TYPE1').send_keys(j) #분류시설 입력
        time.sleep(1)
        driver.find_element_by_id("BTN_FIND_BY_NAME").click() #검색하기 클릭
        time.sleep(1)
        elem=driver.find_element_by_id("statList") # 충전소 검색 목록 가져오기
        if elem.text=='': #목록이없으면 pass
            continue
        else:
            a=elem.text.split('\n') #주차장명, 상태, 급속 여부 나눠서 리스트로 나눠준다.
            for k in range(0,len(a)):
                if k%3==0:
                    title.append(a[k])
                    gu.append(i)
                    facility.append(j)
                    date.append(str(dt.datetime.now()))
                if k%3==1:
                    state.append(a[k])
                if k%3==2:
                    kind.append(a[k])

h_df=pd.DataFrame(data={'구명':gu,'시설분류':facility,'주차장명':title,'사용상태':state,'충전기타입':kind,'수집시간':date})


In [20]:
#환경부데이터 csv로 저장
h_df.to_csv('환경부 크롤링'+str(int(time.time())))

In [27]:
#타기관 데이터 모으기

#드라이버 연결, url 설정
driver=webdriver.Edge("c:/msedgedriver") 
url="https://www.ev.or.kr/evmonitor#"

#웹사이트 이동
driver.get(url)
time.sleep(3)

#팝업창 방지
try:
    result=driver.switch_to_alert()
#     print(result.text)
    #확인
    result.accept()
    #끄기
    result.dismiss()
except: 
    "There is no alert"
    
seoul_list=['강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구','도봉구','동대문구','동작구',\
           '마포구','서대문구','서초구','성동구','성북구','송파구','양천구','영등포구','용산구','은평구','종로구','중구','중랑구']
type_list=['공공시설','주차시설','휴게시설','관광시설','상업시설','차량정비시설','공동주택시설','근린생활시설','교육문화시설','기타시설']
title=[]
state=[]
kind=[]
date=[]
gu=[]
facility=[]

driver.find_element_by_id("F_SI_DO1").send_keys('서울특별시') #서울특별시 입력
time.sleep(1)
driver.find_element_by_id("orgme").click() #환경부체크박스 해제
time.sleep(2)
driver.find_element_by_id("orgetc").click() #타기관 체크박스 클릭
time.sleep(3)


for i in seoul_list:
    driver.find_element_by_id("F_GU_GUN1").send_keys(i) #구입력
    time.sleep(1)
    for j in type_list:
        driver.find_element_by_id('F_STAT_TYPE1').send_keys(j) #분류시설 입력
        time.sleep(3)
        driver.find_element_by_id("BTN_FIND_BY_NAME").send_keys(Keys.ENTER) #검색하기 클릭
        time.sleep(3)
        elem=driver.find_element_by_id("statList") # 충전소 검색 목록 가져오기
        if elem.text=='': #목록이없으면 pass
            continue
        else:
            a=elem.text.split('\n') #주차장명, 상태, 급속 여부 나눠서 리스트로 나눠준다.
            new_a=[i for i in a if "생략" in i] #이하생략을 걸러줄 변수 생성
            if len(new_a)>=1 : #이하생략이 들어갈 경우 미리 삭제
                for m in a:
                    if m == new_a[0]:
                        a.remove(m)
            for k in range(0,len(a)):
                if k%3==0:
                    title.append(a[k])
                    gu.append(i)
                    facility.append(j)
                    date.append(str(dt.datetime.now()))
                if k%3==1:
                    state.append(a[k])
                if k%3==2:
                    kind.append(a[k])

t_df=pd.DataFrame(data={'구명':gu,'시설분류':facility,'주차장명':title,'사용상태':state,'충전기타입':kind,'수집시간':date})

C:\Users\hrim1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use driver.switch_to.alert instead
  if sys.path[0] == '':


In [28]:
#타기관 데이터 csv로 저장
t_df.to_csv('타기관 크롤링'+str(int(time.time())))